In [1]:
# Seldon Deploy SDK
from seldon_deploy_sdk import Configuration, ApiClient, SeldonDeploymentsApi, OutlierDetectorApi, DriftDetectorApi, ModelMetadataServiceApi
from seldon_deploy_sdk.auth import OIDCAuthenticator

# Setup Auth

In [11]:
SD_IP = "34.73.238.47"
config = Configuration()
config.host = f"https://{SD_IP}/seldon-deploy/api/v1alpha1"
config.oidc_client_id = "sd-api"
config.oidc_server = f"https://{SD_IP}/auth/realms/deploy-realm"
config.oidc_client_secret = "sd-api-secret"
config.auth_method = "client_credentials"
config.verify_ssl = "False"

def auth():
    auth = OIDCAuthenticator(config)
    config.id_token = auth.authenticate()
    api_client = ApiClient(configuration=config, authenticator=auth)
    return api_client

# Create Deployment Dictionary

In [15]:
YOUR_NAME = "josh"
DEPLOYMENT_NAME = f"{YOUR_NAME}-test"
NAMESPACE = "seldon"

MODEL_NAME = "inception"
MODEL_LOCATION = "gs://seldon-models/v1.13.1/sklearn/iris"
MODEL_TYPE = "SKLEARN_SERVER"

CANARY_NAME = "simple-cnn"
CANARY_LOCATION = "gs://seldon-models/xgboost/iris"
CANARY_TYPE = "XGBOOST_SERVER"

CPU_REQUESTS = "0.5"
MEMORY_REQUESTS = "1Gi"

CPU_LIMITS = "0.5"
MEMORY_LIMITS = "1Gi"

In [16]:
mldeployment = {
    "kind": "SeldonDeployment",
    "metadata": {
        "name": DEPLOYMENT_NAME,
        "namespace": NAMESPACE
    },
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "spec": {
        "name": DEPLOYMENT_NAME,
        "protocol": "seldon",
        "predictors": [
            {
                "componentSpecs": [
                    {
                        "spec": {
                            "containers": [
                                {
                                    "name": f"{MODEL_NAME}",
                                    "resources": {
                                        "requests": {
                                            "cpu": CPU_REQUESTS,
                                            "memory": MEMORY_REQUESTS
                                        },
                                        "limits": {
                                            "cpu": CPU_LIMITS,
                                            "memory": MEMORY_LIMITS
                                        }
                                    }
                                }
                            ]
                        }
                    }
                ],
                "name": "default",
                "replicas": 1,
                "traffic": 70,
                "graph": {
                    "implementation": MODEL_TYPE,
                    "modelUri": MODEL_LOCATION,
                    "name": f"{MODEL_NAME}",
                    "logger": {
                        "mode": "all"
                    }
                }
            },
            {
                "componentSpecs": [
                    {
                        "spec": {
                            "containers": [
                                {
                                    "name": f"{CANARY_NAME}",
                                    "resources": {
                                        "requests": {
                                            "cpu": CPU_REQUESTS,
                                            "memory": MEMORY_REQUESTS
                                        },
                                        "limits": {
                                            "cpu": CPU_LIMITS,
                                            "memory": MEMORY_LIMITS
                                        }
                                    }
                                }
                            ]
                        }
                    }
                ],
                "name": "canary",
                "replicas": 1,
                "annotations":{
                    "seldon.io/canary":"true"
                },
                "traffic": 30,
                "graph": {
                    "implementation": CANARY_TYPE,
                    "modelUri": CANARY_LOCATION,
                    "name": f"{CANARY_NAME}",
                    "logger": {
                        "mode": "all"
                    }
                    }
                }
        ]
    }
}

Once you have specified your `mldeployment` JSON it is a simple API call to create your deployment. 

In [17]:
deployment_api = SeldonDeploymentsApi(auth())
deployment_api.create_seldon_deployment(namespace=NAMESPACE, mldeployment=mldeployment)

ApiException: (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'content-type': 'application/json', 'date': 'Wed, 15 Jun 2022 18:31:57 GMT', 'content-length': '105', 'x-envoy-upstream-service-time': '1', 'server': 'istio-envoy'})
HTTP response body: {"body":{"code":403,"message":"Authorization Denied","requestId":"78e7c631-895b-42ba-8555-4257e2948843"}}
